In [21]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arseniskobelev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df = pd.read_csv('./data.csv')
df = df[pd.notnull(df['InputData'])]

# remove unused columns
df = df.drop('RawInputData', axis=1)
df = df.drop('DockerImage', axis=1)
df = df.drop('DockerImageVersion', axis=1)
df = df.drop('KubernetesGracePeriodInSeconds', axis=1)
df = df.drop('KubernetesNamespace', axis=1)
df = df.drop('KubernetesObjectName', axis=1)
df = df.drop('Service', axis=1)
df = df.drop('Method', axis=1)
df = df.drop('KubernetesObjectType', axis=1)

print(df.tail(10))

                                             InputData               Function
97   terminate the pod named "tier=database" and wa...  Kubernetes.Delete.Pod
98   delete the pod named "cache-1" in the namespac...  Kubernetes.Delete.Pod
99   remove the pod named "app=logger" in the names...  Kubernetes.Delete.Pod
100  terminate the pod named "env=testing" in the n...  Kubernetes.Delete.Pod
101                          kill kubernetes pod nginx  Kubernetes.Delete.Pod
102                        delete kubernetes pod nginx  Kubernetes.Delete.Pod
103                     delete kubernetes pod database  Kubernetes.Delete.Pod
104                                     kill pod nginx  Kubernetes.Delete.Pod
105                                    kill pod db-pod  Kubernetes.Delete.Pod
106                                   delete pod nginx  Kubernetes.Delete.Pod


In [16]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

df['InputData'] = df['InputData'].apply(clean_text)
print(df.head(10))

                          InputData               Function
0             create kubernetes pod  Kubernetes.Create.Pod
1       could create kubernetes pod  Kubernetes.Create.Pod
2      want create pod inkubernetes  Kubernetes.Create.Pod
3                   want create pod  Kubernetes.Create.Pod
4                        create pod  Kubernetes.Create.Pod
5             pod kubernetes create  Kubernetes.Create.Pod
6             pod create kubernetes  Kubernetes.Create.Pod
7                new kubernetes pod  Kubernetes.Create.Pod
8                new pod kubernetes  Kubernetes.Create.Pod
9  please create new pod kubernetes  Kubernetes.Create.Pod


In [27]:
X = df.InputData
y = df.Function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [92]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

model = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MultinomialNB()),
                  ])
model.fit(X_train, y_train)

functions = ['Kubernetes.Create.Pod', 'Kubernetes.Delete.Pod']

# %%time
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred, target_names=functions))

accuracy 0.9814814814814815


In [134]:
def predict_with_fallback(model_repr, input_data_param, threshold_param, fallback_value):
    # Predict probabilities for the input samples
    proba = model_repr.predict_proba(input_data_param)

    # Get the maximum predicted probability for each sample
    max_proba = np.max(proba, axis=1)

    # Check if the maximum predicted probability is above the threshold
    if np.any(max_proba > threshold_param):
        # At least one sample has a predicted probability above the threshold
        # Return the predictions for those samples
        predictions = model_repr.predict(input_data_param)
        return predictions[0]

    else:
        return fallback_value


In [138]:
input_data = ["I need a new pod"]

threshold = .65

normal_prediction = model.predict(input_data)[0]
prediction_with_threshold = predict_with_fallback(
    model_repr=model,
    input_data_param=input_data,
    fallback_value="General.Output.UnknownCommand",
    threshold_param=threshold
)

print(f"Prediction with threshold: {prediction_with_threshold}")
print(f"Normal prediction: {normal_prediction}")



Prediction with threshold: Kubernetes.Create.Pod
Normal prediction: Kubernetes.Create.Pod
